# Definición de la Red Neuronal

In [2]:
import numpy as np
from random import shuffle
import pickle

In [10]:
class ModeloB:
    
    def __init__(self, dim_in  = 2, h_layers = [3 , 5], dim_out = 2, lr = 0.02):
        np.random.seed(505) #Establecida para experimentación
        self.dim_in  = dim_in # Dimensión de la entrada
        self.dim_h_layers = len(h_layers) # Número de capas ocultas
        self.dim_out = dim_out # Dimensión capa de salida

        self.lr = lr #Learning Reate

        self.layers = h_layers
        
        self.layers.insert(0,dim_in)
        self.layers.append(dim_out)

        self.ident = lambda x : x
        self.tanh = lambda x : np.tanh(x)
        self.soft_max = lambda x : np.exp(x - np.max(x))/np.exp(x - np.max(x)).sum(0, keepdims=True) #Cero porque son vectores columna

        self.weigths = list() #Der*Izq
        self.bias = list()
        self.Hs = list()

        for k in range(self.dim_h_layers + 1):
            self.weigths.append(np.random.rand(self.layers[k+1],self.layers[k])/np.sqrt(self.layers[k+1]))#Pa que converja rápido
            self.bias.append(np.ones((self.layers[k+1],1)))
            self.Hs.append(np.zeros((self.layers[k+1],1)))

    def _backward(self,y_idx,x_idx):
        y = np.zeros((self.sizeVocaculary,1))
        y[y_idx] = 1

        d_salida = self.Hs[-1] - y
        d_hidden =(1 - self.Hs[-2]**2) * np.dot(self.weigths[-1].T,d_salida)
        d_embedd =  np.dot(self.weigths[-2].T,d_hidden)

        #Actualización de pesos
        #self.weigths[-1] -=  self.lr * np.dot(d_salida,self.Hs[-2].T)
        #self.weigths[-2] -=  self.lr * np.dot(d_hidden,self.Hs[-3].T)
        self.weigths[-1] -=  self.lr * np.outer(d_salida,self.Hs[-2])
        self.weigths[-2] -=  self.lr * np.outer(d_hidden,self.Hs[-3])
        self.weigths[-3][:,x_idx] -= self.lr * d_embedd.reshape(-1) 

        #Actualización de bias
        self.bias[-1] = self.bias[-1] - self.lr * d_salida
        self.bias[-2] = self.bias[-2] - self.lr * d_hidden

    def _fordward(self, x):
        #Obtiene el embedding
        self.Hs[0] = self.weigths[0][:,x].reshape((100,1))
        #Capa oculta
        self.Hs[1] = self.tanh(np.dot(self.weigths[1],self.Hs[0]) + self.bias[1])
        #Predicción
        self.Hs[2] = self.soft_max(np.dot(self.weigths[2],self.Hs[1]) + self.bias[2])
        
    def train(self, epochs = 1):
        for i in range(epochs):
            count = 0 
            for k in self.train_split:
                self._fordward(self.bigrams[k][0])
                self._backward(self.bigrams[k][1],self.bigrams[k][0])
                count += 1 
                if (count%1000 == 0):
                  print('Epoca: ',i,' iteración: ',k,' / ',self.sizeData)    
            
    def predic(self, x):
        self._fordward(self.bigrams[k][0])
        
    def loadData(self, bigrams, sizeVocaculary):
        self.bigrams = bigrams
        self.sizeData = len(bigrams)
        self.sizeVocaculary = sizeVocaculary
        self.buildTrainValidationSplits(bigrams, 0.3)
        
    def buildTrainValidationSplits(self,bigrams, validation_split = 0.3):
        indexes = [i for i in range (len (bigrams))]
        #shuffle(indexes) #Mezclamos los datos 
        split = int(len(bigrams)*0.3)
        #self.train = idx_sh[idx_sh:]
        #self.validation = idx_sh[:idx_sh]
        self.train_split = indexes
            
    def loadModel(self,file='model.pk'):
        with open('model.pk','rb') as model_file:
            self.weigths = pickle.load(model_file)
            self.bias = pickle.load(model_file)
        print('¡Modelo cargado correctamente!')
    
    def export(self,file='model_2.pk'):
        with open(file,'wb') as model_file:
            pickle.dump(self.weigths, model_file)
            pickle.dump(self.bias, model_file)
        

In [6]:
with open('Corpus.pk','rb') as corpus_file:
    dic_words = pickle.load(corpus_file)
    dic_index = pickle.load(corpus_file)
    vocabulario = pickle.load(corpus_file)
    bigrams = pickle.load(corpus_file)
    

In [11]:
dim = len(vocabulario)
model = ModeloB(dim, [100 , 300], dim, 0.2)
model.loadData(bigrams, dim)
model.train()

Epoca:  0  iteración:  999  /  28129
Epoca:  0  iteración:  1999  /  28129
Epoca:  0  iteración:  2999  /  28129
Epoca:  0  iteración:  3999  /  28129
Epoca:  0  iteración:  4999  /  28129
Epoca:  0  iteración:  5999  /  28129
Epoca:  0  iteración:  6999  /  28129
Epoca:  0  iteración:  7999  /  28129
Epoca:  0  iteración:  8999  /  28129
Epoca:  0  iteración:  9999  /  28129


KeyboardInterrupt: 

In [ ]:

#model._fordward(5)
#model._backward(1,5)

#for m,h in zip(model.weigths,model.Hs):
    #print(m.shape,h.shape)